#Set Up

In [ ]:
!pip install torchprofile 1>/dev/null
!pip install fast-pytorch-kmeans 1>/dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/food_product_main

In [ ]:
import torchvision.transforms as transforms
import torchvision
import glob
import os
import torch
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import DataLoader

import random
from collections import OrderedDict, defaultdict

import numpy as np
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from tqdm.auto import tqdm
from torchprofile import profile_macs

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

##Dataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Normalize((0, 0, 0), (255, 255, 255))
])

In [ ]:
# Open the text file in read mode
with open('/content/drive/MyDrive/product_label.txt', 'r') as file:
    # Read all lines from the file and store them as a list
    lines = file.readlines()

# Strip newline characters from the end of each line and store them in a new list
class_set = sorted([line.strip() for line in lines])
print(class_set)

['bada_coconut_original', 'banana_choco_pie', 'bichobi_biscuit', 'binch', 'busor_busor_bulgogi', 'butter_waffle', 'chic_choc_chocolate_cookie', 'choco_hazelnut', 'choco_pie_orion', 'choripong_mushmellow', 'cookie_dasse', 'couque_dasse', 'crown_corn_chip', 'crown_cracker', 'custard_real_cream', 'digest_thin', 'dolaon_sun', 'enaak_chicken', 'gu_on_potato', 'homerun_ball', 'honey_butter_chip', 'hureswi_berry_fresh', 'kobuk_chip_mini', 'kokal_corn', 'korae_bab', 'lotte_choco', 'lotte_choco_abc_cookie', 'lotte_sand_original', 'lotte_teok_pie', 'maccaret_original', 'memorie_donut', 'mini_apple_cookie', 'mu_dukduk_potato', 'nongshim_shrimp_chip', 'nongshim_shrimp_chip_hot', 'nooneul_potato', 'oh_yes_mini', 'omma_sun_pie', 'onion_ring_original', 'oreo_stick_choco', 'peanut_sand', 'phokha_chip_onion', 'phokha_chip_original', 'postick', 'slim_potato_chip', 'strawberry_cookie', 'swing_chip_garlic', 'swing_chip_hot', 'tacco', 'white_hazelnut']


In [ ]:
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform, is_each_class):
        self.transform = transform
        self.image_paths = []
        for ext in ['png', 'jpg']:
            if is_each_class:
              self.image_paths += glob.glob(os.path.join(root_dir, f'*.{ext}'))
            else:
              self.image_paths += glob.glob(os.path.join(root_dir, '*', f'*.{ext}'))
        self.class_lbl = { cls: i for i, cls in enumerate(class_set)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = read_image(self.image_paths[idx], ImageReadMode.RGB).float()
        # img = Image.open(self.image_paths[idx]).convert('RGB')
        cls = os.path.basename(os.path.dirname(self.image_paths[idx]))
        label = self.class_lbl[cls]

        return self.transform(img), torch.tensor(label)

##Model

In [ ]:
import torch.nn.functional as F

class VGG(nn.Module):
  ARCH = [32, 64, 'M', 128, 128, 'M', 256, 256, 'M', 256, 512, 'M']

  def __init__(self) -> None:
    super().__init__()

    layers = []
    counts = defaultdict(int)

    def add(name: str, layer: nn.Module) -> None:
      layers.append((f"{name}{counts[name]}", layer))
      counts[name] += 1

    in_channels = 3
    for x in self.ARCH:
      if x != 'M':
        # conv-bn-relu
        add("conv", nn.Conv2d(in_channels, x, 3, padding=1, bias=False))
        add("bn", nn.BatchNorm2d(x))
        add("relu", nn.ReLU(True))
        in_channels = x
      else:
        # maxpool
        add("pool", nn.MaxPool2d(2))

    self.backbone = nn.Sequential(OrderedDict(layers))
    self.classifier = nn.Linear(512, len(class_set))

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.backbone(x)
    x = x.mean([2, 3])
    x = self.classifier(x)
    probabilities = F.softmax(x, dim=1)  # Apply softmax activation to convert logits to probabilities
    confidence_percentages = probabilities * 100     # Convert probabilities to percentages

    return confidence_percentages

In [ ]:
import copy

checkpoint_path = "/content/drive/MyDrive/checkpoint_channel_pruning.pth" ## modify this to checkpoint path
checkpoint = torch.load(checkpoint_path, map_location="cpu")
model = VGG().cuda()
model.load_state_dict(checkpoint)
recover_model = lambda: model.load_state_dict(checkpoint)

##Test 1

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='food_product_main/test', transform=transform)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# Set the model to evaluation mode
model.eval()

# Create an Excel writer object
with pd.ExcelWriter('evaluation_results.xlsx') as writer:

    data = []
    for class_idx, label in enumerate(class_set):
        test_set = CustomDataset(f'food_product_main/test/{label}', transform, True)
        predicted_labels = []
        true_labels = []
        image_ids = []

        # Iterate over the validation set
        for image_id, (inputs, targets) in enumerate(test_set):
            # Move inputs to GPU if available
            inputs = inputs.cuda()

            # Perform inference
            with torch.no_grad():
                outputs = model(inputs.unsqueeze(0))  # Add batch dimension
                predicted_label = outputs.argmax(dim=1).item()

            # Append predicted and true labels to the list
            predicted_labels.append(predicted_label)
            true_labels.append(targets.item())  # Convert tensor to item
            image_ids.append(image_id + 1)  # Image ID starting from 1

        # Convert predicted and true labels to numpy arrays
        predicted_labels = np.array(predicted_labels)
        true_labels = np.array(true_labels)

        # Calculate accuracy
        accuracy = (predicted_labels == true_labels).mean()

        # Store detailed results in a DataFrame
        results = []
        for img_id, true, pred in zip(image_ids, true_labels, predicted_labels):
            results.append([img_id, label, class_set[pred], class_set[true], accuracy])

        df = pd.DataFrame(results, columns=['image_id', 'class_label', 'predicted_label', 'true_label', 'accuracy'])

        # Write detailed results to an Excel sheet
        df.to_excel(writer, sheet_name=f'{label}', index=False)

        # Calculate confusion matrix and classification report
        conf_matrix = confusion_matrix(true_labels, predicted_labels)
        report = classification_report(true_labels, predicted_labels, output_dict=True)

        # Extract precision and recall for the current class
        class_str = str(class_idx)
        if class_str in report:
            precision = report[class_str]['recall']
            support = report[class_str]['support']
            true_positives = int(support * precision)
            false_positives = support - true_positives
            total_images = len(true_labels)  # Total number of images tested for this class
            precision_percentage = f"{precision * 100:.2f}%"

            data.append([class_set[class_idx], total_images, true_positives, false_positives, precision_percentage])
        else:
            data.append([class_set[class_idx], 0, 0, 0, 0.0, 0.0])

    # Create a DataFrame for the summary metrics
    df_summary = pd.DataFrame(data, columns=['Class', 'Test Image', 'True', 'False', 'Precision'])

    # Save the summary metrics to the same Excel file
    df_summary.to_excel(writer, sheet_name='Class_Metrics', index=False)

print("Results saved to evaluation_results.xlsx")


##Test 2

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='food_product_main', transform=transform)
test_set = CustomDataset('food_product_main/test', transform, False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Set the model to evaluation mode
model.eval()

# Create a list to store predicted and true labels
predicted_labels = []
true_labels = []

# Iterate over the validation set
for inputs, targets in test_set:
    # Move inputs to GPU if available
    inputs = inputs.cuda()

    # Perform inference
    with torch.no_grad():
        outputs = model(inputs.unsqueeze(0))  # Add batch dimension
        predicted_label = outputs.argmax(dim=1).item()

    # Append predicted and true labels to the list
    predicted_labels.append(predicted_label)
    true_labels.append(targets.item())

# Convert predicted and true labels to numpy arrays
predicted_labels = np.array(predicted_labels)
true_labels = np.array(true_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Visualize the confusion matrix
# plt.figure(figsize=(10, 8))
# plt.imshow(conf_matrix, cmap='Blues', interpolation='nearest')
# plt.colorbar()
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

# Specify the classes you are interested in (indices of the classes)
specific_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# Generate the classification report as a dictionary
report = classification_report(true_labels, predicted_labels, output_dict=True)

# Prepare data for the DataFrame
data = []

for class_idx in specific_classes:
    class_str = str(class_idx)
    if class_str in report:
        precision = report[class_str]['precision']
        recall = report[class_str]['recall']
        support = report[class_str]['support']
        true_positives = int(support * recall)
        false_negatives = support - true_positives
        total_images = support  # Total number of actual images in this class
        false_positives = sum((true_labels == class_idx) & (predicted_labels != class_idx)) # Number of incorrect predictions for this class

        data.append([class_idx, total_images, true_positives, false_positives, precision, recall])
    else:
        data.append([class_idx, 0, 0, 0, 0.0, 0.0])

# Create a DataFrame
df = pd.DataFrame(data, columns=['class', 'test_image', 'True', 'False', 'precision', 'recall'])

# Save the DataFrame to an Excel file
df.to_excel('class_metrics.xlsx', index=False)

print("Results saved to class_metrics.xlsx")


Results saved to class_metrics.xlsx


#Model Inference Testing

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="model_channel_pruning.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details[0]['shape'])
print(output_details[0]['shape'])
print(input_details)

[  8   3 128 128]
[ 8 50]
[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  8,   3, 128, 128], dtype=int32), 'shape_signature': array([  8,   3, 128, 128], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
# Load and preprocess the image
image_path = "/content/f29f697d-e928-4b73-b43b-0f7333b81e34.jpg"
image = Image.open(image_path)
image = image.resize((input_details[0]['shape'][2], input_details[0]['shape'][3]))
image = np.array(image, dtype=np.float32) / 255 # Normalize pixel values to [0, 1]

image = np.array([image for i in range(8)])
image = np.transpose(image, (0, 3, 1, 2))

index = input_details[0]['index']

# Set input tensor
interpreter.set_tensor(index, image)

# Perform inference
interpreter.invoke()

In [ ]:
# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
# print(output_data[0][2])
predicted_class = np.argmax(output_data)
# print(predicted_class)

# Load labels (if available)
labels = None
labels_path = 'product_label.txt'  # Replace with your label file path
if labels_path:
    with open(labels_path, 'r') as f:
        labels = [line.strip() for line in f.readlines()]

# Post-process inference results
top_classes = output_data[0].argsort()[-5:][::-1]  # Get top 5 classes with highest scores

# Print detected objects and confidence scores
for class_id in top_classes:
    confidence = output_data[0][class_id]
    probabilities = (np.exp(confidence) / np.sum(np.exp(output_data[0]))) * 100
    print("Class: {} - Confidence: {:} - Percentage: {:.2f}%".format(labels[class_id], confidence, probabilities))

Class: tacco - Confidence: 41.797176361083984 - Percentage: 100.00%
Class: chic_choc_chocolate_cookie - Confidence: 7.594037055969238 - Percentage: 0.00%
Class: nongshim_shrimp_chip - Confidence: 6.517587661743164 - Percentage: 0.00%
Class: kobuk_chip_mini - Confidence: 4.528082370758057 - Percentage: 0.00%
Class: banana_choco_pie - Confidence: 3.9245169162750244 - Percentage: 0.00%
